<a href="https://colab.research.google.com/github/Aldana-777/PROGCOM-B/blob/main/Problema_de_Carrera_mejorada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROBLEMA DE CARRER MEJORADA

In [ ]:
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

class Empresa:
    def __init__(self, nombre, ticker, precio_accion, eps, valor_contable, roe):
        self.nombre = nombre
        self.ticker = ticker
        self.precio_accion = precio_accion
        self.eps = eps
        self.valor_contable = valor_contable
        self.roe = roe

    def calcular_pe_ratio(self):
        return self.precio_accion / self.eps

    def calcular_pb_ratio(self):
        return self.precio_accion / self.valor_contable

    def calcular_roe_pct(self):
        return self.roe * 100

class ValuacionEmpresasApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualización de Valoración de Empresas (NASDAQ)")

        self.empresas = self.cargar_datos()

        self.crear_widgets()
        self.anim = None
        self.graficar_metrica_animate("P/E")

    def cargar_datos(self):
        return [
            Empresa("Apple Inc.", "AAPL", 175.00, 6.50, 4.50, 0.28),
            Empresa("Microsoft Corp.", "MSFT", 320.00, 9.20, 15.50, 0.35),
            Empresa("Alphabet Inc.", "GOOGL", 135.00, 5.90, 18.20, 0.17)
        ]

    def crear_widgets(self):
        frame_top = ttk.Frame(self.root)
        frame_top.pack(pady=10)

        self.combo = ttk.Combobox(frame_top, values=["P/E", "P/B", "ROE (%)"], state="readonly", width=10)
        self.combo.set("P/E")
        self.combo.pack(side=tk.LEFT, padx=5)

        btn_actualizar = ttk.Button(frame_top, text="Actualizar Gráfico", command=self.actualizar_grafico)
        btn_actualizar.pack(side=tk.LEFT)

        # Configurar figura y canvas para matplotlib
        plt.style.use('seaborn-darkgrid')
        self.fig, self.ax = plt.subplots(figsize=(7, 5))
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.get_tk_widget().pack()

        # Estilo general del gráfico
        self.ax.tick_params(axis='x', labelrotation=20, labelsize=12)
        self.ax.tick_params(axis='y', labelsize=12)

    def actualizar_grafico(self):
        metrica = self.combo.get()
        # Cancelar animación anterior si existe
        if self.anim:
            self.anim.event_source.stop()
        self.graficar_metrica_animate(metrica)

    def graficar_metrica_animate(self, metrica):
        self.ax.clear()

        nombres = [e.nombre for e in self.empresas]
        valores = []

        if metrica == "P/E":
            valores = [e.calcular_pe_ratio() for e in self.empresas]
        elif metrica == "P/B":
            valores = [e.calcular_pb_ratio() for e in self.empresas]
        elif metrica == "ROE (%)":
            valores = [e.calcular_roe_pct() for e in self.empresas]

        colores = ['#3498db', '#e67e22', '#2ecc71']  # azul, naranja, verde

        bars = self.ax.bar(nombres, [0]*len(valores), color=colores, edgecolor='black', linewidth=1.5)

        self.ax.set_title(f"Métrica: {metrica}", fontsize=18, weight='bold')
        self.ax.set_ylabel(metrica, fontsize=14)
        self.ax.set_ylim(0, max(valores) * 1.3)

        def animate(frame):
            # frame va de 0 a 100
            progress = frame / 100
            for bar, val in zip(bars, valores):
                bar.set_height(val * progress)

            # Mostrar valores arriba de las barras durante la animación
            self.ax.clear()
            bars = self.ax.bar(nombres, [val * progress for val in valores], color=colores, edgecolor='black', linewidth=1.5)
            self.ax.set_title(f"Métrica: {metrica}", fontsize=18, weight='bold')
            self.ax.set_ylabel(metrica, fontsize=14)
            self.ax.set_ylim(0, max(valores) * 1.3)
            self.ax.tick_params(axis='x', labelrotation=20, labelsize=12)
            self.ax.tick_params(axis='y', labelsize=12)

            for bar, val in zip(bars, valores):
                height = bar.get_height()
                self.ax.text(bar.get_x() + bar.get_width() / 2, height + max(valores)*0.02, f"{val:.2f}",
                             ha='center', va='bottom', fontsize=12, fontweight='bold', color='black')

        self.anim = animation.FuncAnimation(self.fig, animate, frames=101, interval=15, repeat=False)
        self.canvas.draw()

if __name__ == "__main__":
    root = tk.Tk()
    app = ValuacionEmpresasApp(root)
    root.mainloop()